# Hyperparameter Tuning using HyperDrive


In [1]:
from azureml.core import Workspace, Experiment
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import ScriptRunConfig, Environment
from azureml.core.compute import ComputeTarget, AmlCompute
import os

In [2]:
#Initialize Workspace & create Experiment
ws = Workspace.from_config()
exp_hyper = Experiment(workspace=ws, name="udacity-project-hyperdrive")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

#run = exp_hyper.start_logging()



Workspace name: quick-starts-ws-251798
Azure region: westeurope
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-251798


In [3]:
found = False
key = "HeartFailurePrediction"
description_text = "Heart Failure Prediction Dataset Kaggle"

if key in ws.datasets.keys(): 
       found = True
       dataset = ws.datasets[key] 

if not found:      
        # Set Kaggle credentials as environment variables or in a separate Kaggle.json file  
        os.environ['KAGGLE_USERNAME'] = '<your_kaggle_username>'  
        os.environ['KAGGLE_KEY'] = '<your_kaggle_api_key>'  
        
        # kaggle datasets download -d fedesoriano/heart-failure-prediction

        # Download the Kaggle dataset using the KaggleDatasetFactory  
        from azureml.data.dataset_factory import KaggleDatasetFactory  

        factory = KaggleDatasetFactory()  
        dataset = factory.create_dataset(  
            dataset_id='<kaggle_dataset_id>',  
            version='<kaggle_dataset_version>'  
        )  

        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                    name=key,
                                    description=description_text)
    
# Convert to Pandas DF
heart_disease_df = dataset.to_pandas_dataframe()
    
heart_disease_df.to_csv('data.csv', index=False)

In [4]:
cluster_name = "udacity-finalproject-compute"

# Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Check if the compute cluster already exists, if not, create a new one
if cluster_name not in ws.compute_targets:
    # Configure the provisioning details for the compute cluster
    provisioning_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2s_v3', max_nodes=4)

    # Create the compute cluster
    compute_target = ComputeTarget.create(ws, cluster_name, provisioning_config)

    # Wait for the compute cluster to be provisioned
    compute_target.wait_for_completion(show_output=True)
else:
    compute_target = ws.compute_targets[cluster_name]

In [8]:
if "training" not in os.listdir():
    os.mkdir("./training")
    
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dependencies.yml')

src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

run = exp_hyper.submit(src)

RunDetails(run).show()

#run.wait_for_completion(show_output=True)


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Hyperdrive Configuration

Using a KNeighborsClaissifier, using RandomParameterSampling with BanditPolicy

In [9]:
# Define the hyperparameter search space  
ps = RandomParameterSampling({  
    '--n_neighbors': choice(7, 9, 12),  
    '--weights': choice('uniform', 'distance'),  
#     '--algorithm': choice('auto', 'ball_tree', 'kd_tree', 'brute'),  
#     '--leaf_size': choice(10, 20, 30),  
#     '--p': choice(1, 2),  
    '--metric': choice('euclidean', 'manhattan', 'chebyshev', 'minkowski')  
})  

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)  

# Create a HyperDriveConfig using estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,  
                                    hyperparameter_sampling=ps,  
                                    primary_metric_name='Accuracy',  
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,  
                                    max_total_runs=20,  
                                    max_concurrent_runs=4)  

In [10]:
# Submit the HyperDrive run to the experiment  
hyperdrive_run = exp_hyper.submit(config=hyperdrive_config)  
  

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
# Use the RunDetails widget to visualize the run details  
RunDetails(hyperdrive_run).show()  

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [13]:
# import joblib

# # Get the best run from the HyperDrive experiment
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])

['--metric', 'minkowski', '--n_neighbors', '12', '--weights', 'uniform']


In [14]:
print(best_run.get_file_names())

['outputs/model.joblib', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']


In [15]:
model = best_run.register_model(model_name='hd_best_model', model_path='outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

